In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense

In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [3]:
LAYERS_COUNT = 4
LSTM_DIM = 256
TEMPERATURE = 0.8

In [4]:
with open("./data/complete_lotr.txt", "rb") as f:
    text = f.read().decode(encoding='utf-8')
vocab = sorted(set(text))
vocab_size = len(vocab)
char_to_idx = {c:i for i,c in enumerate(vocab)}
idx_to_char = {i:c for c,i in char_to_idx.items()}

In [5]:
filepath = "model_weights_saved.hdf5"

testing_model = tf.keras.models.Sequential()
for i in range(LAYERS_COUNT):
    testing_model.add(
            LSTM(
                LSTM_DIM, 
                return_sequences=True if (i!=(LAYERS_COUNT-1)) else False,
                batch_input_shape=(1, 1, vocab_size),
                stateful=True
            )
        )
testing_model.add(Dense(vocab_size, activation = 'softmax'))
adam = tf.keras.optimizers.Adam(lr = 0.01)
testing_model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

testing_model.load_weights(filepath)    

In [6]:
def gumbel_sample(probs, temperature = TEMPERATURE):
  """Helper function to sample an index from a probability array"""
  # from fchollet/keras
  probs = np.asarray(probs).astype('float64')
  probs = np.log(probs) / temperature
  exp_preds = np.exp(probs)
  probs = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, probs, 1)
  return np.argmax(probas)

def predict_next_char(model, current_char):
  x = np.zeros((1, 1, vocab_size))
  x[:,:,char_to_idx[current_char]] = 1
  y = model.predict(x, batch_size=1)
  next_char_idx = gumbel_sample(y[0,:])
  next_char = idx_to_char[next_char_idx]
  return next_char

def generate_text(model, seed, length):
    """Generate characters from a given seed"""
    generated_text = seed
    model.reset_states()
    for c in seed[:-1]:
        next_char = predict_next_char(model, c)
    current_char = seed[-1]

    for i in range(length - len(seed)):
        next_char = predict_next_char(model, current_char)
        current_char = next_char
        generated_text += current_char
    print(generated_text)


In [16]:
#generate_text(testing_model, seed = "The ring", length = 400)
gen_text_length = 200
seed_list = ["On the way to Mordor"]
for temperature in [.1, .8,1.0]:
    for starting in seed_list:
        print(f"Temperature: {temperature} \n")
        generate_text(testing_model, seed = starting, length = gen_text_length)
        print("\n")

        

Temperature: 0.1 

On the way to Mordor before the Shirriff, very scent. The West was ever been the basin miles and pockets called or a right to the way in a great sat more about him and had seen in her later eastward, 


Temperature: 0.8 

On the way to Mordor, and the air no one was desperately. 'Oh I want now!' He said to himself merry.
     But it was a white grey foes of falling like some bullies were looking to the Morgul Towers t


Temperature: 1.0 

On the way to Mordor guessed silence. In thought and the peril across the marrion of the crown, and a whole will that grew hand; and the shadows of the battle was to like the Sun tears out of the sun 


